In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Demographic Data Processing


In [144]:
def remove_minus(x):
    if (x == "-"):
        return "NaN"
    else:
        return x.replace("-", "").replace("+", "").replace(",", "")

In [149]:
#this data is for 11 variables, all census tracts in California, using ACS 2009 5YR estimates 

demog_data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/demographic_data/'
#get percentages of each race
race = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B02001_race_clean.csv')
race["Percent White"] = race["Estimate; White alone"]/race["Estimate; Total:"]
race["Percent Black or African African"] = race["Estimate; Black or African American alone"]/race["Estimate; Total:"]
race["Percent Asian"] = race["Estimate; Asian alone"]/race["Estimate; Total:"]


edu = pd.read_csv(demog_data_path+ 'ACS_09_5YR_S1501_bach_clean.csv')
edu = edu.rename({"Percent of population 25 years and over  with Bachelor\'s degree": "Percent of population 25 years and over with Bachelor\'s degree"}, axis=1)
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].astype(float).dropna()


#get rid of + and ***
hvalue = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25077_hvalue_clean.csv')
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].apply(lambda x: x.replace("+", "").replace("-", "NaN").replace(",", ""))
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].astype(float)

#hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].astype(float).dropna()

#add attached and detached and find the %
single = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25024_singlefam_clean.csv')
single["Total Single Family"] = single["Estimate; 1, attached"]+ single["Estimate; 1, detached"]
single["% Single Family"] = single["Total Single Family"] / single["Estimate; Total:"]
#single["Tract"] = (single["Id2"].astype(str)).apply(lambda x: int(x[4:8]))



#find % Hispanic and non hispanic
hispanic = pd.read_csv(demog_data_path+ 'ACS_17_5YR_B03003_hispanic_clean.csv')
hispanic["% Hispanic or Latino"] = hispanic["Estimate; Total: - Hispanic or Latino"]/hispanic["Estimate; Total:"]
hispanic["% not Hispanic or Latino"] = hispanic["Estimate; Total: - Not Hispanic or Latino"]/hispanic["Estimate; Total:"]
#hispanic["Tract"] = (hispanic["Id2"].astype(str)).apply(lambda x: int(x[4:8]))



poverty = pd.read_csv(demog_data_path+ "ACS_17_5YR_S1701_poverty_clean.csv")
poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"] = poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"] = poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"].astype(float).dropna()
# poverty["Tract"] = (poverty["Id2"].astype(str)).apply(lambda x: int(x[4:8]))

units = pd.read_csv(demog_data_path+'ACS_09_5YR_B25001_units_clean.csv')
#units["Tract"] = (units["Id2"].astype(str)).apply(lambda x: int(x[4:8]))


#find % renter occupied
tenure = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25003_tenure_clean.csv')
tenure["% Renter Occupied"] = tenure["Estimate; Renter occupied"]/tenure["Estimate; Total:"]
tenure["% Owner Occupied"] = tenure["Estimate; Owner occupied"]/tenure["Estimate; Total:"]


income = pd.read_csv(demog_data_path + 'ACS_09_5YR_S1903_income.csv')
income["Median income (dollars); All households"] = income["Median income (dollars); All households"].apply(remove_minus)

income["Median income (dollars); All households"] = income["Median income (dollars); All households"].astype(float)


In [151]:
# single.head()

#### Create Controlled features dataframe

In [4]:
race_units_merged = race.merge(units, how='left', left_on="Id2", right_on="Id2")
print("race and units merged", "length", len(race_units_merged))
#print("race and units merged", "dtypes", race_units_merged.dtypes)

edu_merged = race_units_merged.merge(edu, how='left', left_on="Id2", right_on="Id2")
print("edu merged", "length", len(edu_merged))
#print("edu merged", "dtypes", edu_merged.dtypes)

hispanic_merged = edu_merged.merge(hispanic, how='left', left_on="Id2", right_on="Id2")
print("hispanic merged", "length", len(hispanic_merged))
#print("hispanic merged", "dtypes", hispanic_merged.dtypes)

income_merged = hispanic_merged.merge(income, how='left', left_on="Id2", right_on="Id2")
print("income merged", "length", len(income_merged))
#print("income", "dtypes", income_merged.dtypes)

poverty_merged = income_merged.merge(poverty, how='left', left_on="Id2", right_on="Id2")
print("poverty merged", "length", len(poverty_merged))
#print("poverty", "dtypes", poverty_merged.dtypes)

hvalue_merged = poverty_merged.merge(hvalue, how='left', left_on="Id2", right_on="Id2")
print("hvalue merged", "length", len(hvalue_merged))
#print("hvalue", "dtypes", hvalue_merged.dtypes)

single_merged = hvalue_merged.merge(single, how='left', left_on="Id2", right_on="Id2")
print("single merged", "length", len(single_merged))
#print("singe merged", "dtypes", single_merged.dtypes)

# merged = single_merged.merge(tenure, how='left', left_on="Id2", right_on="Id2")
# print("tenure merged", "length", len(merged))


merged = single_merged.merge(tenure, how='left', left_on="Id2", right_on="Id2")
print("tenure merged", "length", len(merged))


race and units merged length 7049
edu merged length 7049
hispanic merged length 7049
income merged length 7049
poverty merged length 7049
hvalue merged length 7049
single merged length 7049
tenure merged length 7049


In [148]:
single_merged

,Id2,Geography_x,Estimate; Total:_x,Margin of Error; Total:_x,Estimate; White alone,Margin of Error; White alone,Estimate; Black or African American alone,Margin of Error; Black or African American alone,Estimate; American Indian and Alaska Native alone,Margin of Error; American Indian and Alaska Native alone,...,Margin of Error; Median value (dollars),Geography_y,Estimate; Total:,Margin of Error; Total:,"Estimate; 1, detached","Margin of Error; 1, detached","Estimate; 1, attached","Margin of Error; 1, attached",Total Single Family,% Single Family
0,6001400100,"Census Tract 4001, Alameda County, California",2872,233,2224,257,119,102,6,15,...,***,"Census Tract 4001, Alameda County, California",1439,28,1006,94,307,82,1313,0.912439
1,6001400200,"Census Tract 4002, Alameda County, California",2076,165,1650,217,47,49,11,21,...,39846,"Census Tract 4002, Alameda County, California",932,25,577,73,43,32,620,0.665236
2,6001400300,"Census Tract 4003, Alameda County, California",4964,369,3792,320,557,172,36,47,...,70431,"Census Tract 4003, Alameda County, California",2801,49,1234,158,107,76,1341,0.478758
3,6001400400,"Census Tract 4004, Alameda County, California",4223,553,3097,668,401,194,0,132,...,51801,"Census Tract 4004, Alameda County, California",2020,73,1007,136,115,56,1122,0.555446
4,6001400500,"Census Tract 4005, Alameda County, California",3759,308,1756,256,1316,327,14,24,...,45234,"Census Tract 4005, Alameda County, California",1735,50,673,98,98,59,771,0.444380
5,6001400600,"Census Tract 4006, Alameda County, California",1676,200,642,156,793,147,4,8,...,55299,"Census Tract 4006, Alameda County, California",784,42,422,80,74,55,496,0.632653
6,6001400700,"Census Tract 4007, Alameda County, California",3901,548,1424,357,1807,369,0,132,...,65391,"Census Tract 4007, Alameda County, California",2016,140,998,160,156,93,1154,0.572421
7,6001400800,"Census Tract 4008, Alameda County, California",3005,335,1049,231,1138,237,0,132,...,86200,"Census Tract 4008, Alameda County, California",1678,54,612,109,98,61,710,0.423123
8,6001400900,"Census Tract 4009, Alameda County, California",2667,392,1103,223,1454,437,0,132,...,59493,"Census Tract 4009, Alameda County, California",1198,49,539,113,51,44,590,0.492487
9,6001401000,"Census Tract 4010, Alameda County, California",6152,727,1428,391,3499,533,0,132,...,43211,"Census Tract 4010, Alameda County, California",2682,85,1201,176,50,46,1251,0.466443


In [14]:
merged.columns
merged = merged[["Id2", 'Percent of population 25 years and over with Bachelor\'s degree', '% Hispanic or Latino', 
      'Percent below poverty level; Estimate; Population for whom poverty status is determined', 
      '% Single Family', '% Owner Occupied', 'Percent Black or African African', 'Percent White', 'Percent Asian',
      'Estimate; Total Number of Housing Units', 'Estimate; Median value (dollars)', 
        'Median income (dollars); All households']]
merged = merged.rename({'Percent below poverty level; Estimate; Population for whom poverty status is determined': "% below poverty level",
              'Estimate; Total Number of Housing Units': 'Total number of housing units',
              'Estimate; Median value (dollars)': "Median home value",
              'Median income (dollars); All households': "Median income"})
print("num tracts", len(merged))
      
merged.head()

num tracts 7049


,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0
1,6001400200,37.6,0.088633,5.5,0.665236,0.657428,0.022640,0.794798,0.069364,932,909500.0,122647.0
2,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0
3,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0
4,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0


#### Get only Bay Area counties

In [6]:
county_codes = ['6001', '6013', '6041', '6055', '6075', '6081', '6085', '6097', '6095']

In [13]:
merged_bay = merged[merged["Id2"].apply(lambda x: str(x)[:4]).isin(county_codes)]
print("num tracts", len(merged_bay))
merged_bay.head()


num tracts 1405


,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0
1,6001400200,37.6,0.088633,5.5,0.665236,0.657428,0.022640,0.794798,0.069364,932,909500.0,122647.0
2,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0
3,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0
4,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0


#### Merge Demographic features with neighbors dataframe to contribute CRA eligibility information by tract and exclude CTs without oppositely coded neighbors

Universe: all of the CTs in the 9 county bay area that had loan activity have oppositely coded neighbors. The boolean_tracts_all_2009.csv file includes all the tracts that have oppositely coded neighbors and their CRA eligibility.



In [18]:
data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/'
cra_elig_neighbors = pd.read_csv(data_path+ 'boolean_tracts_all_2009.csv')
cra_elig_neighbors.head()

,Tract,Year,CRA Eligible
0,6001433700,2009,eligible
1,6001435101,2009,not eligible
2,6001435101,2009,not eligible
3,6001435101,2009,not eligible
4,6001435101,2009,not eligible


In [17]:
demog_with_cra = pd.merge(merged, cra_elig_neighbors, how='right', left_on='Id2', right_on='Tract')
#.drop("Id2", axis=1) 
#i think that this is supposed to
#be a right merge because the demographics dataframe has all the census tracts, while the boolean neighbors one only
#has the census tracts in the bay area that have neighbors
demog_with_cra = demog_with_cra.drop(["Tract", "Year"], axis=1)
print(len(demog_with_cra))
demog_with_cra.head()



1228


,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA Eligible
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0,not eligible
1,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0,not eligible
2,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0,not eligible
3,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0,eligible
4,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,586700.0,39802.0,eligible


#### Get total number and percentages of loans data from HMDA files
This is currently for 2009, but next step is to do it for the average of the recovery period 2013-2017


Universe: all of the CTs in the 9 county bay area that had loan activity during 2009

In [110]:
filepath = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data/'
Alameda = pd.read_csv(filepath+'Alameda_2009_parsed.csv')
Alameda["Geoid"] = Alameda["Tract"].apply(lambda x: int(x*100 + 6001000000))

ContraCosta = pd.read_csv(filepath+'ContraCosta_2009_parsed.csv')
ContraCosta["Geoid"] = ContraCosta["Tract"].apply(lambda x: int(x*100 + 6013000000))

Napa = pd.read_csv(filepath+'Napa_2009_parsed.csv')
Napa["Geoid"] = Napa["Tract"].apply(lambda x: int(x*100 + 6055000000))

Marin = pd.read_csv(filepath+'Marin_2009_parsed.csv')
Marin["Geoid"] = Marin["Tract"].apply(lambda x: int(x*100 + 6041000000))

SanMateo = pd.read_csv(filepath+'SanMateo_2009_parsed.csv')
SanMateo["Geoid"] = SanMateo["Tract"].apply(lambda x: int(x*100 + 6081000000))

SanFrancisco = pd.read_csv(filepath+'SanFrancisco_2009_parsed.csv')
SanFrancisco["Geoid"] = SanFrancisco["Tract"].apply(lambda x: int(x*100 + 6075000000))

Solano = pd.read_csv(filepath+'Solano_2009_parsed.csv')
Solano["Geoid"] = Solano["Tract"].apply(lambda x: int(x*100 + 6095000000))

SantaClara = pd.read_csv(filepath+'SantaClara_2009_parsed.csv')
SantaClara["Geoid"] = SantaClara["Tract"].apply(lambda x: int(x*100 + 6085000000))

Sonoma = pd.read_csv(filepath+'Sonoma_2009_parsed.csv')
Sonoma["Geoid"] = Sonoma["Tract"].apply(lambda x: int(x*100 + 6097000000))

counties = [Alameda, ContraCosta, Napa, Marin, SanMateo, SanFrancisco,Solano,SantaClara, Sonoma]

bay_counties_loans = pd.concat(counties).reset_index(drop=True)
bay_counties_loans["Total Loans"] = bay_counties_loans['# HI borrower, LI tract'] + bay_counties_loans['# HI borrower, HI tract']+ bay_counties_loans['# LI borrower, LI tract']
+ bay_counties_loans['# LI borrower, HI tract']
bay_counties_loans = bay_counties_loans.drop(["Tract", "Year"], axis=1)


#### Merge Loans Data with cra_demog features

In [111]:
allvars_2009 = pd.merge(demog_with_cra, bay_counties_loans, how="left", left_on='Id2', right_on='Geoid').drop(['CRA Eligible_y', "Geoid"], axis=1)
allvars_2009['CRA'] = allvars_2009['CRA Eligible_x'].apply(lambda x: 1 if x == 'eligible' else 0)
allvars_2009.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/all_vars2009.csv")

allvars_2009.head()

,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,...,"% HI borrower, LI tract","# HI borrower, LI tract","% HI borrower, HI tract","# HI borrower, HI tract","% LI borrower, LI tract","# LI borrower, LI tract","%LI borrower, HI tract","# LI borrower, HI tract",Total Loans,CRA
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,...,0.000000,0.0,0.981818,54.0,0.0,0.0,0.0,0.0,54.0,0
1,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,...,0.000000,0.0,0.906250,58.0,0.0,0.0,0.0,0.0,58.0,0
2,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,...,0.000000,0.0,0.979167,47.0,0.0,0.0,0.0,0.0,47.0,0
3,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,...,0.877193,50.0,0.000000,0.0,0.0,0.0,0.0,0.0,50.0,1
4,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,...,0.870968,27.0,0.000000,0.0,0.0,0.0,0.0,0.0,27.0,1


#### Regression Data Prep

In [118]:
allvars_2009.columns
noinc = ['% HI borrower, LI tract',
       '# HI borrower, LI tract', '% HI borrower, HI tract',
       '# HI borrower, HI tract', '% LI borrower, LI tract',
       '# LI borrower, LI tract', '%LI borrower, HI tract',
       '# LI borrower, HI tract', 'Geoid', 'Total Loans', "CRA Eligible_x", "type", "County"]

X_vars = allvars_2009.loc[:, ~allvars_2009.columns.isin(noinc)]
y = allvars_2009["Total Loans"]
print(len(y))
print(len(X_vars))
X_vars.head()

1228
1228


,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0,0
1,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0,0
2,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0,0
3,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0,1
4,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,586700.0,39802.0,1


In [119]:
# This is the number of nans (missing data) 
for col in X_vars.columns:
    num = sum(np.isnan(X_vars[col]))
    print(col, num)
    
    

Id2 0
Percent of population 25 years and over with Bachelor's degree 1
% Hispanic or Latino 205
Percent below poverty level; Estimate; Population for whom poverty status is determined 205
% Single Family 1
% Owner Occupied 1
Percent Black or African African 1
Percent White 1
Percent Asian 1
Estimate; Total Number of Housing Units 0
Estimate; Median value (dollars) 17
Median income (dollars); All households 1
CRA 0


#### Some of the data is Nan, I dealt with this is 3 ways, discuss the best strategy for this with Carolina

Strategy 1: drop all the Nans, lose ~20% of the data

In [121]:
#need to concatenate x with y, dropna and then unconcatenate in order to do this correctly
xydf = X_vars.copy()
xydf["y"] = y
dropped = xydf.dropna()

X_drop = dropped.loc[:, dropped.columns != 'y']
#y_drop = dropped['y']

print("num tracts", len(X_drop))
print("y", len(y_drop))
X_drop.head()

num tracts 1015
y 1015


,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0,0
1,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0,0
2,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0,0
3,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0,1
4,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,586700.0,39802.0,1


Strategy 2: infill nans with mean of column

In [124]:
X_vars_mean = X_vars.fillna(X_vars.mean())
y_mean = y.fillna(y.mean())
print("num tracts", len(X_vars))
print("num y", len(y))
X_vars_mean.head()


num tracts 1228
num y 1228


,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0,0
1,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0,0
2,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0,0
3,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0,1
4,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,586700.0,39802.0,1


Strategy 3: infill nans with median of column

In [125]:
X_vars_med = X_vars.fillna(X_vars.median())

y_med = y.fillna(y.median())
print("X", len(X_vars_med))
print("y", len(y_med))

X_vars_med.head()


X 1228
y 1228


,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0,0
1,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0,0
2,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0,0
3,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0,1
4,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,586700.0,39802.0,1


#### Data Normalization

In [126]:
def normalize_columns(data, mean_df, std_df):
    '''
    Input:
      data (data frame): contains only numeric columns
    Output:
      data frame, the same data, except each column is standardized 
      to have 0-mean and unit variance
    '''
    normalized_data=(data-mean_df.mean())/std_df.std()

    return normalized_data

In [127]:
def unnormalize_columns(data, mean_df, std_df):
    unnormalized_data=(data*std_df.std())+mean_df.mean()

    return unnormalized_data

#### Regression Using dropped NaNs method

In [130]:
normal_x_drop = normalize_columns(X_drop, X_drop, X_drop)
normal_y_drop = normalize_columns(y_drop, y_drop, y_drop)

In [132]:
from sklearn import linear_model
sk_model = linear_model.LinearRegression()
sk_model.fit(normal_x_drop,normal_y_drop)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [135]:
from collections import OrderedDict
linear_coef_dict = {}
for linear_coef, linear_feature in zip(sk_model.coef_, X_vars.columns):
    linear_coef_dict[linear_feature] = linear_coef
    
sorted_coefs_linear = OrderedDict(sorted(linear_coef_dict.items(), key=lambda t: t[1]))
sorted_coefs_linear


OrderedDict([('Percent White', -0.27258307357582173),
             ('Estimate; Median value (dollars)', -0.1802384889463188),
             ('Percent Asian', -0.17616035990482332),
             ('Percent Black or African African', -0.10267521594433242),
             ('Percent below poverty level; Estimate; Population for whom poverty status is determined',
              -0.06864439726389242),
             ('% Hispanic or Latino', -0.029225514400186996),
             ('CRA', 0.03409852616665652),
             ('Id2', 0.06423751338118262),
             ("Percent of population 25 years and over with Bachelor's degree",
              0.07497825769341768),
             ('% Owner Occupied', 0.11329052393443738),
             ('% Single Family', 0.19409543212601302),
             ('Median income (dollars); All households', 0.23685152316952895),
             ('Estimate; Total Number of Housing Units', 0.5322966677423042)])

#### Regression using filled means method

In [136]:
normal_x_vars_mean = normalize_columns(X_vars_mean, X_vars_mean, X_vars_mean)
normal_y_mean = normalize_columns(y_mean, y_mean, y_mean)

In [137]:
from sklearn import linear_model
sk_model_mean = linear_model.LinearRegression()
sk_model_mean.fit(normal_x_vars_mean,normal_y_mean)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [138]:
from collections import OrderedDict
linear_coef_dict = {}
for linear_coef, linear_feature in zip(sk_model_mean.coef_, X_vars.columns):
    linear_coef_dict[linear_feature] = linear_coef
    
sorted_coefs_linear_mean = OrderedDict(sorted(linear_coef_dict.items(), key=lambda t: t[1]))
sorted_coefs_linear_mean

OrderedDict([('Percent White', -0.2406589297300957),
             ('Percent Asian', -0.1627241507290869),
             ('Percent Black or African African', -0.08579536499837401),
             ('Estimate; Median value (dollars)', -0.0520897204611604),
             ('% Hispanic or Latino', -0.041761699127489955),
             ("Percent of population 25 years and over with Bachelor's degree",
              -0.004286297721727586),
             ('Id2', 0.006293545345081028),
             ('Percent below poverty level; Estimate; Population for whom poverty status is determined',
              0.038605134907601465),
             ('CRA', 0.05931227882930579),
             ('% Single Family', 0.12143575349566846),
             ('% Owner Occupied', 0.15492160410474418),
             ('Median income (dollars); All households', 0.19571858449749396),
             ('Estimate; Total Number of Housing Units', 0.7309329991940453)])

#### Regression using filled medians method


In [139]:
normal_x_median = normalize_columns(X_vars_med, X_vars_med, X_vars_med)
normal_y_median = normalize_columns(y_med, y_med, y_med)

In [141]:
from sklearn import linear_model
sk_model_med = linear_model.LinearRegression()
sk_model_med.fit(normal_x_median,normal_y_median)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [142]:
from collections import OrderedDict
linear_coef_dict = {}
for linear_coef, linear_feature in zip(sk_model_med.coef_, X_vars.columns):
    linear_coef_dict[linear_feature] = linear_coef
    
sorted_coefs_linear_med = OrderedDict(sorted(linear_coef_dict.items(), key=lambda t: t[1]))
sorted_coefs_linear_med

OrderedDict([('Percent White', -0.24446619437416256),
             ('Percent Asian', -0.16614787094237304),
             ('Percent Black or African African', -0.08943402260210127),
             ('Estimate; Median value (dollars)', -0.0536007519352044),
             ('% Hispanic or Latino', -0.04588823441702904),
             ("Percent of population 25 years and over with Bachelor's degree",
              -0.004639249603893194),
             ('Id2', 0.006075829490589675),
             ('Percent below poverty level; Estimate; Population for whom poverty status is determined',
              0.03750944422438277),
             ('CRA', 0.06221351509162301),
             ('% Single Family', 0.12226360834152093),
             ('% Owner Occupied', 0.1568035455704303),
             ('Median income (dollars); All households', 0.1950521834862294),
             ('Estimate; Total Number of Housing Units', 0.7319505968590125)])